In [1]:
import contextvars
from contextvars import Context, ContextVar, copy_context
from contextlib import contextmanager
import functools
from typing import Callable, Any, Optional
import hashlib
import uuid
import base64
import time
from datetime import datetime, timezone
import json
import pprint
from pathlib import Path

from pydantic import BaseModel, Field

In [2]:
datetime.fromtimestamp(datetime.now().timestamp())

datetime.datetime(2023, 5, 29, 14, 23, 7, 835368)

In [3]:
datetime.fromtimestamp(datetime.utcnow().replace(tzinfo=timezone.utc).timestamp())

datetime.datetime(2023, 5, 29, 14, 23, 7, 976007)

In [4]:
datetime.fromtimestamp(datetime.utcnow().timestamp())

datetime.datetime(2023, 5, 29, 12, 23, 8, 69599)

In [5]:
datetime.fromtimestamp(datetime.now().timestamp(), tz=timezone.utc)

datetime.datetime(2023, 5, 29, 12, 23, 8, 226310, tzinfo=datetime.timezone.utc)

In [6]:
datetime.fromtimestamp(datetime.now(timezone.utc).timestamp(), tz=timezone.utc)

datetime.datetime(2023, 5, 29, 12, 23, 8, 397808, tzinfo=datetime.timezone.utc)

In [7]:
datetime.fromtimestamp(datetime.utcnow().timestamp()).replace(tzinfo=timezone.utc).astimezone(tz=None)

datetime.datetime(2023, 5, 29, 14, 23, 8, 512799, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CEST'))

In [8]:
datetime.fromtimestamp(time.time())

datetime.datetime(2023, 5, 29, 14, 23, 8, 634415)

In [9]:
type(datetime.now(timezone.utc).timestamp())

float

In [10]:
uid_seed = uuid.uuid1().bytes + uuid.uuid4().bytes
print(uid_seed)
print(len(uid_seed))
uid_bytes = hashlib.sha256(uid_seed).digest()
print(uid_bytes)
print(len(uid_bytes))
uid_hex = hashlib.sha256(uid_seed).hexdigest()
print(uid_hex)
print(len(uid_hex))
print(bytes.fromhex(uid_hex))
# uuid_base64 = base64.urlsafe_b64encode(uid_bytes).decode("utf-8").rstrip("=")
# print(uuid_base64)
# print(len(uuid_base64))
# print(base64.urlsafe_b64decode(uuid_base64 + "=="))


b'\x92(\xf0\xbe\xfe\x1b\x11\xed\x94\xc24.\xb7\xcd>\x8f\xb3\x8d\xad\x02\x9a2A+\x82x?1u\xb9<\x05'
32
b"\xec\xbb\x02\x98\xferl\xa2F\xff\xe4di/\x9a\x04\xd1$\x9f\x89\xf46\x9e7{'\xbd\x04T\x80\xa2\x9b"
32
ecbb0298fe726ca246ffe464692f9a04d1249f89f4369e377b27bd045480a29b
64
b"\xec\xbb\x02\x98\xferl\xa2F\xff\xe4di/\x9a\x04\xd1$\x9f\x89\xf46\x9e7{'\xbd\x04T\x80\xa2\x9b"


In [11]:
def get_random_hash() -> str:
    return hashlib.sha256(uuid.uuid1().bytes + uuid.uuid4().bytes).hexdigest()

In [12]:
def get_utc_timestamp() -> float:
    return datetime.now(timezone.utc).timestamp()


class TraceNode(BaseModel):
    name: str
    id: str = Field(default_factory=get_random_hash)
    parent: Optional['TraceNode'] = None
    nb_children: int = 0
    timestamp_utc: float = Field(default_factory=get_utc_timestamp)
    metadata: Optional[dict] = None

    def update(self, name: str, metadata: Optional[dict] = None) -> tuple['TraceNode', 'TraceNode']:
        parent = TraceNode(
            name=self.name,
            id=self.id,
            parent=self.parent,
            nb_children=self.nb_children + 1,
            timestamp_utc=self.timestamp_utc,
            metadata=self.metadata
        )
        child = TraceNode(
            name=name,
            parent=parent,
            metadata=metadata,
        )
        return parent, child
    
    def get_root(self) -> 'TraceNode':
        if self.parent is None:
            return self
        return self.parent.get_root()

    def __str__(self) -> str:
        return repr(self)

    def __repr__(self) -> str:
        # id_repr = self.id.hex()[:4]
        id_repr = self.id[:4]
        if self.parent is None:
            return f"(name={self.name} id={id_repr} nb_children={self.nb_children})"
        return f"(name={self.name} id={id_repr} nb_children={self.nb_children} parent={self.parent!r})"
    
    class Config:
        frozen = True


TRACE_CONTEXT = ContextVar[TraceNode](
    'STACK_CONTEXT',
    default=TraceNode(name='root')
)

LOG_DIR = Path('./logs')


# @contextmanager
# def new_context(ctx: ContextModel):
#     orig_token = TRACE_CONTEXT.set(TRACE_CONTEXT.get() + [ctx])
#     try:
#         yield
#     finally:
#         TRACE_CONTEXT.reset(orig_token)


@contextmanager
def update_context(name: str, metadata:Optional[dict] = None):
    parent, child = TRACE_CONTEXT.get().update(name, metadata=metadata)
    TRACE_CONTEXT.set(parent)
    orig_token = TRACE_CONTEXT.set(child)
    try:
        yield
    finally:
        TRACE_CONTEXT.reset(orig_token)


def log_trace(msg: str) -> None:
    trace = TRACE_CONTEXT.get()
    root_trace = trace.get_root()
    root_time = datetime.fromtimestamp(root_trace.timestamp_utc).strftime("%Y%m%d_%H%M%S")
    root_dir_name = f"{root_trace.name}_{root_time}_{root_trace.id}"
    log_dir = LOG_DIR / root_dir_name
    log_dir.mkdir(parents=True, exist_ok=True)
    log_file = log_dir / f"{trace.id}.log.json"
    log_dct = trace.dict(exclude_none=True)
    log_dct['log_message'] = msg
    with log_file.open('w') as f:
        json.dump(log_dct, f)



def trace(func: Callable):
    @functools.wraps(func)
    def trace_decorator(*args, **kwargs):
        # Do something before
        with update_context(name=func.__name__):
            value = func(*args, **kwargs)
            # Do something after
            return value
    return trace_decorator

In [13]:
@trace
def do_a():
    print(TRACE_CONTEXT.get())
    log_trace("Hello world 123!")

@trace
def do_b():
    do_a()
    do_a()

@trace
def do_c():
    do_b()
    do_b()
    do_a()


do_c()
print(TRACE_CONTEXT.get())

(name=do_a id=af35 nb_children=0 parent=(name=do_b id=c152 nb_children=1 parent=(name=do_c id=300e nb_children=1 parent=(name=root id=812d nb_children=1))))
(name=do_a id=9fd4 nb_children=0 parent=(name=do_b id=c152 nb_children=2 parent=(name=do_c id=300e nb_children=1 parent=(name=root id=812d nb_children=1))))
(name=do_a id=ea07 nb_children=0 parent=(name=do_b id=e1b8 nb_children=1 parent=(name=do_c id=300e nb_children=2 parent=(name=root id=812d nb_children=1))))
(name=do_a id=56f9 nb_children=0 parent=(name=do_b id=e1b8 nb_children=2 parent=(name=do_c id=300e nb_children=2 parent=(name=root id=812d nb_children=1))))
(name=do_a id=e749 nb_children=0 parent=(name=do_c id=300e nb_children=3 parent=(name=root id=812d nb_children=1)))
(name=root id=812d nb_children=1)
